In [5]:
import torch 
device = 'cuda' if torch.cuda.is_available else 'cpu'
print(device)
block_size=8
batch_size=4
import torch.nn as nn
from torch.nn import functional as F
learning_rate = 3e-3
eval_iters = 250
drop_out = 0.2
#These are 2 hyperparameters that are really imp

cuda


In [6]:
with open('Wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(len(chars))
print(chars)
vocab_size=len(chars)
print(vocab_size)
#an encoder will convert each element of this array into a number

80
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
80


In [7]:
#Now we will map each character to a number so now any string we encode will be encoded in the above pattern of numbers

string_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l : ''.join([int_to_string[i] for i in l])

encoded_paritosh = encode("paritosh")
print(encoded_paritosh)
decoded_paritosh = decode(encoded_paritosh)
print(decoded_paritosh)

[69, 54, 71, 62, 73, 68, 72, 61]
paritosh


In [8]:
### Listing some basic Torch Data Structures for reference:

 # Tensor: multi-dimensional array that can hold various types
 # Dataset: classes and its derivatives
 

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])
print(vocab_size)

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])
80


In [10]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]
print(len(data))

232310


We are going to split the data in validation and training splits

Why can't we just use the entire data set and train on that?
- So we basically have 18% train and 20% validation (e.g.) because we want the model to generate text that is ***like*** the corpus, not ***exactly***.
- If we just train the model on the entire corpus then the model will just simple memorize all of it.

Why is it a BIGRAM LANGUAGE MODEL

- There are probabilities to predict what the next character in the word is based on the previoud one
- e.g. if the word if "Hello", then
  start of content -> H, H->e, e->l, l->l, l->o.
- only gonna consider the previous chat to predict the next, therefore 2 = bi = bigram

Let us begin by taking a block size = 5

- ... [5, 67, 21, 58, 40] 35, .... (these are the encoded texts from corpus)
- ... 5 [67, 21, 58, 40, 35], ....
- these are prediction and targets, what the model would understand is that 67 is following 5
- We look at how far is the prediction away from the target, then we can optimize for reducing that error

In [10]:
# block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("input is ", context, ", the target is ", target)

input is  tensor([1]) , the target is  tensor(1)
input is  tensor([1, 1]) , the target is  tensor(28)
input is  tensor([ 1,  1, 28]) , the target is  tensor(39)
input is  tensor([ 1,  1, 28, 39]) , the target is  tensor(42)
input is  tensor([ 1,  1, 28, 39, 42]) , the target is  tensor(39)
input is  tensor([ 1,  1, 28, 39, 42, 39]) , the target is  tensor(44)
input is  tensor([ 1,  1, 28, 39, 42, 39, 44]) , the target is  tensor(32)
input is  tensor([ 1,  1, 28, 39, 42, 39, 44, 32]) , the target is  tensor(49)


So basically this is sequencial. We can't expect real time results if the processing is sequencial. Therefore we need batches of these blocks to process in parallel for any real time results from our bigram model.

Hence, we need GPUs

In [11]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+ block_size+1] for i in ix])
    x, y = x.to(device), y.to(device) #pushing it to GPU
    return x, y
    
x, y = get_batch('train')
print("inputs : ", x)
print("targets : ", y)

inputs :  tensor([[64, 62, 67, 60,  1, 54, 65, 65],
        [58, 58, 71, 59, 74, 65, 65, 78],
        [ 1, 54, 67, 57,  1, 61, 62, 72],
        [73, 78,  1, 68, 59,  1, 73, 61]], device='cuda:0')
targets :  tensor([[62, 67, 60,  1, 54, 65, 65,  1],
        [58, 71, 59, 74, 65, 65, 78, 11],
        [54, 67, 57,  1, 61, 62, 72,  1],
        [78,  1, 68, 59,  1, 73, 61, 58]], device='cuda:0')


### How does Gradient Descent Work (recap)
- Basically you take the ***derivative*** of the loss function which has the parameter we need to find. 
- If there are multiple parameters you need to find, you take ***derivative*** of the loss function with ***respect to*** all these parameters.
- When you have 2 or more derivatives of the same function, they are called a **Gradient**.
- We then use this ***Gradient*** to ***descend*** to the lowest point in the **Loss Function**
- When the slope is far from 0, we must take longer steps since we are far from the optimal value, but as we're near 0, we need to take baby steps
- step_size = slope*learning_rate ----- and therefore ---- new_intercept = old_intercept-step_size
- we also need to limit it by the number of steps

In [32]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k]=loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [29]:
class Bigram_LM(nn.Module):
    #when we use the nn.module function, they are learnable parameters
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) #this is just a look-up table for the predictions of what comes after what probabilistically

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else :
            B, T, C =logits.shape
            logits = logits.view(B*T, C) # view just puts a tensor back together. a = a.view(x, y, z)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        # pretty much a probability distribution e.g. [0.167, 0.33, 0.5] are logits, they might represent ab, ac, ad. 
        # Logits are the raw, unnormalized predictions produced by a neural network's output layer before they are transformed into probabilities.
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self.forward(index)
            #focus only on the last time step
            logits = logits[:, -1, :] #becomes B, c , also, -1 will cause it to loop back to the last element
            #apply softmax to get probabilities normalised
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append sampled index to the running sequence
            index = torch.cat((index, index_next), dim = 1) # (B, T+1)
        return index
    
model = Bigram_LM(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


X1Gs5
JFrgyK.&58hKse9oL;!1JQOecQ]ip(u7n,GuX*cvy53L7)MLYt,;N
 wL7XbOyoV".QJ;Cnu'1q;[C3ZN.i3-!qz7]h_g3dzj&:e)vcgUQR)f&HZdK)_bfzwNEl9oV_HL7uQuKD8NGzff'n7E6".Rs*At(,G YUb(20mE_RIks9zhr5c"V&PV3pqqHa;KWt.4rsTKGHLft]ua8wi:)P9F YE4RPmbg]X.RPecKRQX'.hQRQdxd8N- V!W1
E6rd;(deuZzXu7)&Fy[GW0u-dCfs5JbW)CVb;MFlGtjUQRI3kaW6A?Q&g&YqvMC6Ey(sEXbwIM(fdMQegq7TqY*FHZL-B(0Mp_0.0";ISF:o]wL7 myiyjxz2,GH4
&RQZ'C"Oecg648LkkKobYXBCvKtpAvbjtPD*yyYdFl'C,i"2ldBBpY*FyWmyHZOm6qY4?QF7lVidiY!-u7uyyoHD0P
m,4c'b7X*'-FokBCn8qvaksKTs


### Let me recap the above code cell :

Let us Split the cell into 5 parts : 
### 1. __Initialization__ (__init__):
#### a. Parameters 
- vocab_size = number of unique tokens the model can work with (chars & words)

#### b. Embedding Table :
- Initializes an embedding layer by using "nn.Embedding"
- This layer is essentially a look-up table for word embeddings (probabilities)
- It is used to transform discrete categorical data into continuous representations so that we can use this data for ML models to work with.
- Why? In NLP words and tokens are typically represented as discrete values, like integers. e.g "cat" can be represented as 43.
- Using these integers can be problematic, since they don't capture relationships and meanings between words, they treat words as independent and unrelated.
- An Embedding table is basically a lookup table where each row corresponds to a unique token (words in a vocab) and each row contains a continuous vector rep (embedding) of that token.
- Usage? During training of a neural network, you can use these embeddings as a part of the input data.
- The network learns to adjust these embeddings during training to improve the performance of the task at hand.

### 2. __Forward Pass (forward method)__ :
#### a. Parameters
- index = (tensor) represents a sequence of tokens
- targets = (optional tensor) target tokens for training
- They are used to compute the loss during training

#### b. Logits
- concept used especially used in classification and NNs.
- They are raw, unnormalized scores or valies produced by last layer of NN before getting transformed into probabilities 
- They are a way to quantify how strongly the model believes a given input belongs to each class
- In classification problems, logits are used to make decisions about which class an input most likely belongs to.
- They provide a level of abstraction that allows the model to learn complex patterns in the data and assign confidence scores to different classes

### 3. Generate method (text generation):
#### a. Parameters 
- index = representing the unique tokens in our corpus
- max_new_tokens = max num of tokens to generate

#### b. function
- inside the loop for **max_new_tokens** iterations, it repeatedly predicts the next token based on the previous tokens.
- Each iteration uses ***forward*** method to obtain the logits for the next token
- It selects the logits for the last time step (current prediction) and applies a softmax function to convert them into probabilities
- It samples the next token index from the probability distribution using ***torch.multinomial***
- The sample index is appended to the running sequence
- ***This is how a text is generated***

### 4. Model and Context : 
- Instance of Bigram_LM is created with a specific vocab_size and it is moved to the specified 'device'.
- context is initialized as a tensor with zeros representing the starting point for text generation

In [38]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
max_iters =10000
for iter in range(max_iters):

    if iter%1000 ==0:
        losses = estimate_loss()
        print(f'step: {iter}, loss {losses}')

    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, loss {'train': tensor(2.4516), 'val': tensor(2.4752)}
step: 1000, loss {'train': tensor(2.4113), 'val': tensor(2.4894)}
step: 2000, loss {'train': tensor(2.4401), 'val': tensor(2.4814)}
step: 3000, loss {'train': tensor(2.4505), 'val': tensor(2.4904)}
step: 4000, loss {'train': tensor(2.4226), 'val': tensor(2.4731)}
step: 5000, loss {'train': tensor(2.4045), 'val': tensor(2.4939)}
step: 6000, loss {'train': tensor(2.4038), 'val': tensor(2.4870)}
step: 7000, loss {'train': tensor(2.4256), 'val': tensor(2.4735)}
step: 8000, loss {'train': tensor(2.4345), 'val': tensor(2.4642)}
step: 9000, loss {'train': tensor(2.4276), 'val': tensor(2.4965)}
2.252755880355835


## What optimization? 
Appears to be a part of training loop for a machine learning model.

### 1. __Optimizer Initialization__
- initialize optimizer, specifically ***AdamW***
- model.parameters() is passed which provides it with model's ***trainable parameters ***(weights, biases of nn)
- ***lr = learning rate***, set to a specific value which controls the ***step size*** during the gradient descent

### 2. __Training Loop :__
- set max_iters to desired value
- start a loop that will interate over the training data multiple times to train the model

### 3. __Data Batch Retrieval (training data):__
- xb, yb are the training indexes and tragets we need to train the data

### 4. __Forward Pass and Loss Calculation :__
- you call the foward method to calculate the models logits and losses
- ***logits*** represent the models ***raw prediction scores***
- ***loss*** represents the calculated loss, typically a measure of the difference bw the model's prediction and the the actual target.

### 5. __Gradient Calculation & Backprop :__
- before updating the model's parameters, you set the gradients of all model parameters to zero
- Then you compute the gradients of the loss with respect to the model's parameters using ***loss.backward()***. This step is a part of the ***backpropagation*** algo and calculates how much each parametere should be adjusted to minimize loss.

### 6. __Parameter update:__
- optimizer.step()
- finally you update the model's parameters based on the computed gradients and the learning rate 

In [37]:
context = torch.zeros((1, 1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

print(generated_chars)




"I squin ncevetsthe
Cand. thin and winef pore anver. skacel y  sant or tond sque upend-me shiro guesouroffe,  t,"inowonseant aimowofred my

iare itoum tofilin Whed tho mastof bos d

ghilt'loonkoorod ndealll?"Wind hor,

  had T w Deall borang heryoplale g on
Hebs aven'ltimy. r, d he b-thed the sed 95dofok, d witirlfathoury, han, don sha athavey The hemas
othe wag  iza hesoma flle, ablld, t, heorgings bue
tlcac. cedan too Whre " a eth nd wecor--p, Dotshef GI cha plesof kysofus ag trew awhagourki


## Transformers

A Transformer has 4 basic functions : 
- Word Embedding
- Positional Encoding
- Self-Attention
- Residual Connections

--------------------------------------------------------------------------------------------------

- Since a transformer is a type of NN, and NNs only have numbers for input values, hence we need to encode and decode the words
- Therefore ***Word Embedding*** : use a relatively simple NN, that has 1 input for every word or symbol in the vocab that we want to use.
- We call each input a ***Token***
- The inputs are then connected to an **Activation Function**.
- ![Alt text](image-1.png)
- All of these *weights* are determined using ***BackPropagation***.
- ***Positional Encoding*** : a technique that transformers use to keep track of word order.
- We can use sin and cos waves to all these positional encodings
- ![Alt text](image-2.png)
- but, for e.g. "The **Pizza** came out of the **oven** and **it** tasted good", the transformers must associate the "it" with "pizza".
- ***Self-Attention*** calcs similatiry of word and each other word
- One way to calc ***similarity*** is called the **Dot Product**.
- ![Alt text](image-3.png)

- But we already had 2 values, why do we need more ? The new ***Self-Attention*** values for each word contain input from all the other words, and this helps give each word context.
-------------------------------------------------------------------------------

- Instead of this process being sequential, the Transformer can do all of the computation for each word in the input at the same time
- This is why we need a GPU
- ![Alt text](image-4.png)
----------------------------------------------

Now that we are done with the encoder, we need to maintain a relative relation between the encoder and the decoder.
- we find the similarities between each word in the encoder to each word in the decoder using ***dot product***
- ![Alt text](image-5.png)
- Then scale these values using the ***Softmax*** percentages.
- Then add the pairs of scaled values together to get the ***Encoder-Decoder Attention*** values.
- We then run these residual connection values through a ***Fully connected layer***
- ![Alt text](image-6.png)
